<a href="https://colab.research.google.com/github/tmatzxzone/multi-tools/blob/main/Multi_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount Google Drive




In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

##Streamlink

In [ ]:
#@markdown <center><h2>Streamlink Installer</h2></center><br>
#@markdown <center>Install StreamLink</center>
from IPython.display import clear_output
!pip install streamlink
clear_output()

In [ ]:
#@markdown <center><h3><h2>Stream Recorder</h2>Record any stream</h3></center><br>
#@markdown 1- Go to your Google Drive and create a directory called "stream" (whitout quotes).
#@markdown <br />2- Mount Gdrive above.
#@markdown <br />3- Install Streamlink above.
#@markdown <br />4- Start recording bellow.
import os
import time
import subprocess
from IPython.display import clear_output
RUN_Dir = "/content/tools/stream"
 
if not RUN_Dir:
  RUN_Dir = "tools/stream"
os.makedirs(RUN_Dir, exist_ok=True)

#code to use plugin that was removed from streamlink, remove the comment bellow and set the .py name
#if not os.path.exists(f"{RUN_Dir}/.py"):
#  !cd {RUN_Dir} && wget https://raw.githubusercontent.com/limontec/streamlink/master/src/streamlink/plugins/.py

#@markdown ---
#@markdown ####Type stream url
streamURL =  ""  # @param {type:"string"}
#@markdown ---
#@markdown ####Type any name for the stream
name = ""  # @param {type:"string"}
#@markdown ---
#@markdown ####Settings
#@markdown start stream record
start_record = True #@param {type:"boolean"}
#@markdown try restart record after stream disconnect
loop = False #@param {type:"boolean"}
#@markdown upload video file to your Drive after finish every record
upload_record = True #@param {type:"boolean"}
#@markdown upload all videos in this session to your Drive
upload_all = False #@param {type:"boolean"}
#@markdown ---
#@markdown #####Sites supported: https://streamlink.github.io/plugin_matrix.html
#@markdown ---

def record():
  tempo = time.strftime('_%d-%m-%Y_%H-%M-%S')
  if start_record:
    !cd {RUN_Dir} && streamlink --plugin-dirs {RUN_Dir} {streamURL} best -o {RUN_Dir}/{name}{tempo}.mp4 --retry-streams 30 --retry-max 300
  upload(tempo)

def upload(tempo):
  if(os.path.exists('/content/drive')):
    if (upload_record and os.path.exists(RUN_Dir+'/'+name+tempo+'.mp4')) or loop:
      try:
        if(os.path.exists(RUN_Dir+'/'+name+tempo+'.mp4')):
          print("Copying "+name+tempo+".mp4 to drive...")
          !cd {RUN_Dir} && cp {name}{tempo}.mp4 /content/drive/My\ Drive/stream
        else:
          print("No file.")
      except Exception:
        print("Error!")
    elif upload_all:
      try:  
        print("Copying *.mp4 to drive...")
        os.system('cd '+RUN_Dir+' && cp *.mp4 /content/drive/My\ Drive/stream')
      except Exception:
        print("Error!")
    else:
      print("Nothing copied.")
  else:
    print("You need to mount Gdrive to /content/drive to upload")

record()
while loop:
  tempo = time.strftime('_%d-%m-%Y_%H-%M-%S')
  record()
  clear_output()


## Mega

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← MEGA Login</font>
# @markdown <p>Please log in to MEGA first (only needed to use the Uploader).</p>
# ================================================================ #

from functools import wraps
import errno
import os
import signal
import subprocess
import shlex

class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

if not os.path.exists("/root/.ipython/mixlab.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/mixlab.py \
                    -O /root/.ipython/mixlab.py"
    run(split(shellCmd))
from mixlab import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)

# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# Enter MEGA credential
USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please enter your MEGA credential.")

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] MEGA Downloader</font>
URL = "" #@param {type:"string"}
OUTPUT_PATH = "/content/drive/My Drive/" #@param {type:"string"}
# @markdown > URL: is the MEGA link you want to download (ex: mega.nz/file/file_link#decryption_key)</br>OUTPUT_PATH: is where to store the downloaded file(s) (ex: /content/drive/My Drive/)
# ================================================================ #

import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/" \
                "MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
)

if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
transfare()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] MEGA Uploader</font>
# Simple_torrent = False  # @param{type: "boolean"}
# Peerflix = False  # @param{type: "boolean"}
PATH_TO_FILE = "" # @param {type:"string"}
# @markdown > PATH_TO_FILE is the location of the file you want to upload located at. (ex: /content/downloads/file-to-upload.zip)
# ================================================================ #

import time
import subprocess
import contextlib
from IPython.display import clear_output

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    cmd = ""
    if Simple_torrent:
        cmd = ['mega-put', 'downloads', '/colab']
    elif Peerflix:
        cmd = ['mega-put', 'peerflix', '/colab']
    else:
        cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Please log into your MEGA account first!")

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] YouTube-DL</font>
Archive = False
# ================================================================ #

import os, uuid, urllib.parse
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files

Links = widgets.Textarea(placeholder='''Video/Playlist Link
(one link per line)''')

VideoQ = widgets.Dropdown(options=["Best Quality (VP9 upto 4K)", "Best Compatibility (H.264 upto 1080p)"])

AudioQ = widgets.Dropdown(options=["Best Quality (Opus)", "Best Compatibility (M4A)"])

Subtitle = widgets.ToggleButton(value=True, description="Subtitle", button_style="info", tooltip="Subtitle")

SavePathYT = widgets.Dropdown(options=["/content", "/content/downloads"])

AudioOnly = widgets.ToggleButton(value=False, description="Audio Only", button_style="", tooltip="Audio Only")

Resolution = widgets.Select(options=["Highest", "4K", "1440p", "1080p", "720p", "480p", "360p", "240p", "144p"], value="Highest")

Extension = widgets.Select(options=["mkv", "webm"], value="mkv")

UsernameYT = widgets.Text(placeholder="Username")

PasswordYT = widgets.Text(placeholder="Password")

SecAuth = widgets.Text(placeholder="2nd Factor Authentication")

VideoPW = widgets.Text(placeholder="Video Password")

GEOBypass = widgets.Dropdown(options=["Disable", "Hide", "AD", "AE", "AF", "AG", "AI", "AL", "AM", "AO", "AQ", "AR", "AS", "AT", "AU", "AW", "AX", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BL", "BM", "BN", "BO", "BQ", "BR", "BS", "BT", "BV", "BW", "BY", "BZ", "CA", "CC", "CD", "CF", "CG", "CH", "CI", "CK", "CL", "CM", "CN", "CO", "CR", "CU", "CV", "CW", "CX", "CY", "CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "EH", "ER", "ES", "ET", "FI", "FJ", "FK", "FM", "FO", "FR", "GA", "GB", "GD", "GE", "GF", "GG", "GH", "GI", "GL", "GM", "GN", "GP", "GQ", "GR", "GS", "GT", "GU", "GW", "GY", "HK", "HM", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IM", "IN", "IO", "IQ", "IR", "IS", "IT", "JE", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KP", "KR", "KW", "KY", "KZ", "LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", "MD", "ME", "MF", "MG", "MH", "MK", "ML", "MM", "MN", "MO", "MP", "MQ", "MR", "MS", "MT", "MU", "MV", "MW", "MX", "MY", "MZ", "NA", "NC", "NE", "NF", "NG", "NI", "NL", "NO", "NP", "NR", "NU", "NZ", "OM", "PA", "PE", "PF", "PG", "PH", "PK", "PL", "PM", "PN", "PR", "PS", "PT", "PW", "PY", "QA", "RE", "RO", "RS", "RU", "RW", "SA", "SB", "SC", "SD", "SE", "SG", "SH", "SI", "SJ", "SK", "SL", "SM", "SN", "SO", "SR", "SS", "ST", "SV", "SX", "SY", "SZ", "TC", "TD", "TF", "TG", "TH", "TJ", "TK", "TL", "TM", "TN", "TO", "TR", "TT", "TV", "TW", "TZ", "UA", "UG", "UM", "US", "UY", "UZ", "VA", "VC", "VE", "VG", "VI", "VN", "VU", "WF", "WS", "YE", "YT", "ZA", "ZM", "ZW"])

ProxyYT = widgets.Text(placeholder="Proxy URL")

MinSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Min:")

MaxSleep = widgets.BoundedIntText(value=0, min=0, max=300, step=1, description="Max:")

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def upload_archive():
  if ask_yes_no("Do you already have an archive file? (y/n)", default="", interrupt=""):
    try:
      display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Please upload an archive from your computer.</h2><br>"))
      UploadConfig = files.upload().keys()
      clear_output(wait=True)
      if len(UploadConfig) == 0:
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
      elif len(UploadConfig) == 1:
        for fn in UploadConfig:
          if os.path.isfile("/content/" + fn):
            get_ipython().system_raw("mv -f " + "\"" + fn + "\" /root/.youtube-dl.txt && chmod 666 /root/.youtube-dl.txt")
            AudioOnly.observe(AudioOnlyChange)
            Subtitle.observe(SubtitleChange)
            AudioQ.observe(AudioQChange)
            ShowYT()
          else:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
      else:
        for fn in UploadConfig:
          get_ipython().system_raw("rm -f " + "\"" + fn + "\"")
        return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
    except:
      clear_output(wait=True)
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))
  else:
    get_ipython().system_raw("touch '/root/.youtube-dl.txt'")
    AudioOnly.observe(AudioOnlyChange)
    Subtitle.observe(SubtitleChange)
    AudioQ.observe(AudioQChange)
    ShowYT()

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content", "/content/downloads"]

def AudioOnlyChange(change):
  if change["type"] == "change" and change["new"]:
    VideoQ.disabled = True
    Subtitle.disabled = True
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = True
    Extension.options = ["best", "aac", "flac", "mp3", "m4a", "opus", "vorbis", "wav"]
    Extension.value = "best"
    AudioOnly.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    VideoQ.disabled = False
    Subtitle.disabled = False
    if Subtitle.value:
      Subtitle.button_style = "info"
    else:
      Subtitle.button_style = ""
    Resolution.disabled = False
    if AudioQ.value == "Best Quality (Opus)":
      Extension.options = ["mkv", "webm"]
    else:
      Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"
    AudioOnly.button_style = ""

def SubtitleChange(change):
  if change["type"] == "change" and change["new"]:
    Subtitle.button_style = "info"
  elif change["type"] == "change" and change["new"] == False:
    Subtitle.button_style = ""

def AudioQChange(change):
  if change["type"] == "change" and change["new"] == "Best Quality (Opus)":
    Extension.options = ["mkv", "webm"]
    Extension.value = "mkv"
  elif change["type"] == "change" and change["new"] == "Best Compatibility (M4A)":
    Extension.options = ["mkv", "mp4", "webm"]
    Extension.value = "mkv"

def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  display(widgets.HBox([widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      widgets.HTML("<b style=\"color:#888888;\">For website that require an account:</b>"), UsernameYT, PasswordYT, SecAuth, VideoPW,
                                      widgets.HTML("<b><a href=\"https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Officially_assigned_code_elements\" target=\"_blank\">GEO Bypass Country:</a></b>"), GEOBypass,
                                      widgets.HTML("<b style=\"color:#888888;\">Proxy:</b>"), ProxyYT,
                                      widgets.HTML("<b style=\"color:#888888;\">Sleep Interval (second):</b>"), MinSleep, MaxSleep]),
                        widgets.VBox([widgets.HTML("<b style=\"color:#888888;\">Video Quality:</b>"), VideoQ, widgets.HTML("<b style=\"color:#888888;\">Resolution:</b>"), Resolution,
                                      widgets.HTML("<b style=\"color:#888888;\">Audio Quality:</b>"), AudioQ, widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Options:</b>"), widgets.HBox([Subtitle, AudioOnly]),
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), ExtraArg])]), HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"),
          SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
#    display(HTML("*If you want to save in Google Drive please run the cell below."))
    display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    # Account Check
    if UsernameYT.value.strip() and PasswordYT.value.strip():
      accountC = "--username \"" + UsernameYT.value + "\" --password \"" + PasswordYT.value + "\""
    else:
      accountC = ""
    if SecAuth.value.strip():
      secauthC = "-2 " + SecAuth.value
    else:
      secauthC = ""
    if VideoPW.value.strip():
      videopwC = "--video-password " + VideoPW.value
    else:
      videopwC = ""
    # Proxy
    if ProxyYT.value.strip():
      proxyytC = "--proxy " + ProxyYT.value
    else:
      proxyytC = ""
    # GEO Bypass
    if GEOBypass.value == "Disable":
      geobypass = ""
    elif GEOBypass.value == "Hide":
      geobypass = "--geo-bypass"
    else:
      geobypass = "--geo-bypass-country " + GEOBypass.value
    # Video Quality
    if VideoQ.value == "Best Quality (VP9 upto 4K)":
      videoqC = "webm"
    else:
      videoqC = "mp4"
    # Audio Quality
    if AudioQ.value == "Best Quality (Opus)":
      audioqC = "webm"
    else:
      audioqC = "m4a"
    # Audio Only Check
    if AudioOnly.value:
      subtitleC = ""
      thumbnailC = ""
      extC = "-x --audio-quality 0 --audio-format " + Extension.value
      codecC = "bestaudio[ext=" + audioqC + "]/bestaudio/best"
    else:
      if Subtitle.value:
        subtitleC = "--all-subs --convert-subs srt --embed-subs"
      else:
        subtitleC = ""
      if Extension.value == "mp4":
        thumbnailC = "--embed-thumbnail"
      else:
        thumbnailC = ""
      extC = "--merge-output-format " + Extension.value
      if Resolution.value == "Highest":
        codecC = "bestvideo[ext=" + videoqC + "]+bestaudio[ext=" + audioqC + "]/bestvideo+bestaudio/best"
      else:
        codecC = "bestvideo[ext=" + videoqC + ",height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio[ext=" + audioqC + "]/bestvideo[height<=" + Resolution.value.replace("4K", "2160").replace("p", "") + "]+bestaudio/bestvideo+bestaudio/best"
    # Archive
    if os.path.isfile("/root/.youtube-dl.txt"):
      archiveC = "--download-archive \"/root/.youtube-dl.txt\""
    else:
      archiveC = ""
    # Sleep Interval
    if MinSleep.value > 0 and MaxSleep.value > 0:
      minsleepC = "--min-sleep-interval " + MinSleep.value
      maxsleepC = "--max-sleep-interval " + MaxSleep.value
    else:
      minsleepC = ""
      maxsleepC = ""
    # Extra Arguments
    extraargC = ExtraArg.value
    for Link in Links.value.splitlines():
      clear_output(wait=True)
      Count += 1
      display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Processing link " + str(Count) + " out of " + Total + "</h3>"))
      if "youtube.com" in Link or "youtu.be" in Link:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading...</h3><br>"), YouTubeVideo(Link, width=640, height=360), HTML("<br>"))
      else:
        display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">Currently downloading <a href=\"" + Link + "\">" + Link + "</a></h3><br>"))
      if ("youtube.com" in Link or "youtu.be" in Link) and "list=" in Link:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(playlist)s/%(playlist_index)s - %(title)s.%(ext)s" "$Link"
      else:
        !youtube-dl -i --no-warnings --yes-playlist --add-metadata $accountC $secauthC $videopwC $minsleepC $maxsleepC $geobypass $proxyytC $extC $thumbnailC $subtitleC $archiveC $extraargC -f "$codecC" -o "/root/.YouTube-DL/%(title)s.%(ext)s" "$Link"
      if not os.path.exists(SavePathYT.value):
        get_ipython().system_raw("mkdir -p -m 666 " + SavePathYT.value)
      get_ipython().system_raw("mv /root/.YouTube-DL/* '" + SavePathYT.value + "/'")
    # Archive Download
    if os.path.isfile("/root/.youtube-dl.txt"):
      files.download("/root/.youtube-dl.txt")
    ShowYT()

if not os.path.isfile("/usr/local/bin/youtube-dl"):
  get_ipython().system_raw("rm -rf /content/sample_data/ && mkdir -p -m 666 /root/.YouTube-DL/ && apt-get install atomicparsley && curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl && chmod a+rx /usr/local/bin/youtube-dl")
if Archive:
  upload_archive()
else:
  AudioOnly.observe(AudioOnlyChange)
  Subtitle.observe(SubtitleChange)
  AudioQ.observe(AudioQChange)
  ShowYT()

## ZippyShare Downloader

In [ ]:
# ============================= FORM ============================= #
# @markdown <h3>← Install the Module and Dependencies</h3>
# ================================================================ #

import os
from IPython.display import clear_output

!git clone https://github.com/mcrapet/plowshare.git /content/plowshare
clear_output()
%cd plowshare
clear_output()
!make install
clear_output()
!plowmod --install
clear_output()
! apt-get install nodejs

path1 = '/content/downloads'
path2 = '/content/downloads/zippyshare'

if os.path.exists(path1) == False:
  os.makedirs(path1)
  if os.path.exists(path2) == False:
    os.makedirs(path2)
  elif os.path.exists(path2) == True:
    None
elif os.path.exists(path1) == True:
  if os.path.exists(path2) == False:
    os.makedirs(path2)
  elif os.path.exists(path2) == True:
    None

clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <h3>← [Start] zippyshare Downloader</h3></br>
mode = 'single' #@param ["single", "batch"]
# @markdown ---
direct_url = "" #@param {type:"string"}
store_path = "" #@param {type:"string"}
# @markdown > This downloader isn't working as it can't read from zippyshare's weird url (www(random_number).zippyshare)
# ================================================================ #

import os
from IPython.display import clear_output
from google.colab import files

if mode == 'single':
  if direct_url == '':
    print("The URL field is empty!")
  else:
    if store_path == '':
      store_path = '/content/downloads/zippyshare'
      !plowdown {direct_url} -o {store_path}
    else:
      !plowdown {direct_url} -o {store_path}
elif mode == 'batch':
  print("Upload a download.txt file that contains a list of zippyshare links.\n")
  files.upload()
  clear_output()
  if store_path == '':
    store_path = '/content/downloads/zippyshare'
    !plowdown {direct_url} -o {store_path}
  else:
    !plowdown {direct_url} -o {store_path}

## HandBrake

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Install HandBrake</font>
# ================================================================ #

from IPython.display import clear_output
from os import makedirs

makedirs("/content/temp/HandbrakeTemp", exist_ok = True)

!wget -qq https://github.com/vot/ffbinaries-prebuilt/releases/download/v4.2.1/ffmpeg-4.2.1-linux-64.zip 
!rm -f ffmpeg-4.2.1-linux-64.zip
!add-apt-repository ppa:stebbins/handbrake-releases -y 
!apt-get install -y handbrake-cli

clear_output()

In [ ]:
##################################################
#
# Code author: SKGHD
# https://github.com/SKGHD/Handy
#
##################################################

# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] HandBrake</font>
MODE = "SINGLE" #@param ["SINGLE", "BATCH"]
# @markdown > Select mode (batch conversion / single file)
# @markdown ---
SOURCE = "" # @param {type:"string"}
DESTINATION = "" # @param {type:"string"}
FORMAT = "mkv" # @param ["mp4", "mkv"]
RESOLUTION = "480p" # @param ["480p", "576p", "720p", "1080p"]
Encoder = "x264" # @param ["x264", "x265"]
Encoder_Preset = "ultrafast" # @param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
CQ = 30 #@param {type:"slider", min:10, max:30, step:1}
# @markdown > Choose Constant Quality Rate (higher quality / smaller file size)
Additional_Flags = "" # @param {type:"string"}
# ================================================================ #

import smtplib
import os

formats = ('.mkv','.mp4','.ts','.avi','.mov','.wmv')

######## Renames the file ########
def fileName(fPath):
        tName = fPath.split('/')[-1]        
        if tName.endswith('ts'):
          tName = '[HandBrake] ' + tName[:-3] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        else:
          tName = '[HandBrake] ' + tName[:-4] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'    
        return tName

def set_resolution():
  global w,h,flags
  if RESOLUTION == "480p":
    w, h = "854" , "480"
  if RESOLUTION == "480p":
    w, h = "1024" , "576"
  elif RESOLUTION == "720p":
    w, h = "1280" , "720"
  elif RESOLUTION=="1080p":
    w, h = "1920" , "1080"

def addFlags():
  global flags
  flags = f" --encoder {Encoder}  --all-audio -s '0,1,2,3' --cfr --optimize --quality={CQ} --width={w} --height={h} --format={FORMAT} --encoder-preset={Encoder_Preset} "
  if Additional_Flags != "":
    flags += str(Additional_Flags)

set_resolution()
addFlags()

##### HandBrake and Rclone #####
def runner(path):
  f_name = fileName(path)
  hTemp=f"/content/temp/HandbrakeTemp/{f_name}"
  !HandBrakeCLI -i "$path" -o "$hTemp" $flags


  if os.path.isfile(hTemp):
    print(f"\n\n********** Successfully converted {f_name}\n Now saving to Destination.....")
    if os.path.exists('/usr/bin/rclone'):
      !rclone move "$hTemp" --user-agent "Mozilla" "$DESTINATION" --transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40
    else:
      dest = DESTINATION+'/'+f_name
      !mv "$hTemp" "$dest"
  if os.path.isfile(DESTINATION+ '/' +f_name):    
      print(f"\n\n********** Successfully saved {f_name} to Destination")

########## Check Mode ########
if MODE=="BATCH":
  os.makedirs(DESTINATION, exist_ok=True)
  if SOURCE.endswith('/'):
      pass
  else: SOURCE +='/'
  filesList = os.listdir(SOURCE+'.')
  if os.path.isfile(SOURCE+'processed_db.txt'):
    pass
  else:
    with open((SOURCE+'processed_db.txt'), 'w') as fb:
      fb.write("Do not delete this file until all files have been processed!\n")
      fb.close()
  with open((SOURCE+'processed_db.txt'), "r+") as filehandle:
    processedList = [x.rstrip() for x in filehandle.readlines()]

    print('<<<<<<<<<<<<<<<<<< Starting Conversion in Batch mode. >>>>>>>>>>>>>>>>>>')

    for currentFile in filesList:
      if currentFile.endswith(formats):
        if currentFile not in processedList:
          currentPath = SOURCE + currentFile      
          print(f'\n\n**************** Current File to process: {currentFile}')
          runner(currentPath)
          filehandle.write(currentFile+'\n')
    filehandle.close()
        

else:
    if SOURCE.endswith(formats):    
        runner(SOURCE)
    else: print("Are you sure you have selected the correct file?")

## AnonFiles Uploader

In [ ]:
# ============================= FORM ============================= #
# @markdown <p><font size=5px>← [Start] Upload to anonfiles</font></p>
file_path = "" # @param {type: "string"}

url = "https://api.anonfiles.com/upload"
# ================================================================ #

import requests

x = requests.post(url, files = {'file': open(file_path,'rb')},)

print("Download link: " + x.json()["data"]["file"]["url"]["full"])

## Create / Extract Archive

In [ ]:
# ============================= FORM ============================= #
# @markdown <p><font size=5px>← Install the Tools</font></p>
# ================================================================ #

import os
import IPython
from IPython.display import clear_output

os.system("sudo apt update")
os.system("apt install p7zip-full p7zip-rar unrar rar")

clear_output()

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5px>← Create archive</font>
source_path = "/content/drive/My Drive/" #@param {type:"string"}
archive_type = "rar" #@param ["zip", "7z", "rar", "tar", "tar.gz"]
archive_name = "" #@param {type:"string"}
#@markdown > If left empty, the default name will be used (archive)
archive_password = "" #@param {type:"string"}
#@markdown > Leave this field empty if you do not want to protect the archive with password.
compression_level = "no_compression" #@param ["no_compression", "fastest", "fast", "normal", "maximum", "ultra"]
output_path = "" #@param {type:"string"}
#@markdown > If left empty, the default path will be used (/content)

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

from IPython.display import HTML, clear_output
import os, sys, re


if archive_name == "":
    archive_name = "archive"
else:
    pass

if archive_password == "":
    pass
else:
    archive_password = "-p" + archive_password

if compression_level == "no_compression":
    compression_level = "-mx=0"
elif compression_level == "fastest":
    compression_level = "-mx=1"
elif compression_level == "fast":
    compression_level = "-mx=3"
elif compression_level == "normal":
    compression_level = "-mx=5"
elif compression_level == "maximum":
    compression_level = "-mx=7"
elif compression_level == "ultra":
    compression_level = "-mx=9"

if output_path == "":
    output_path = "/content"
else:
    pass


if archive_type == "zip":
    if source_path == "":
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The source_path field is empty!</h2><br></center>"))
    else:
        #output_file_path = re.search("^[\/].+\/", source_path)
        #output_file_path_raw = output_file_path.group(0)
        #delsplit = re.search("\/(?:.(?!\/))+$", source_path)
        #folder_name = re.sub("^[\/]", "", delsplit.group(0))

        #os.environ['inputDir'] = source_path
        #os.environ['outputPath'] = output_file_path_raw
        #os.environ['folderName'] = folder_name
        #os.environ['archiveLevel'] = compression_level
        #os.environ['archivePassword'] = archive_password

        #!7z a -tzip "$archiveLevel" "$archivePassword" "$outputPath"/"$folderName".zip "$inputDirectory"
        !7z a -tzip "$compression_level" "$archive_password" "$output_path"/"$archive_name".zip "$source_path"
else:
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ More archive format will be added in the future.</h2><br></center>"))


if automatically_clear_cell_output is True:
	clear_output()
else:
	pass

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5px>← Extract archive</font>
archive_path = "/content/drive/My Drive/" #@param {type:"string"}
archive_type = "zip" #@param ["zip", "7z", "rar", "tar", "gzip", "iso"]
archive_password = "" #@param {type:"string"}
#@markdown > Leave the archive_password field empty if archive is not password protected.
output_path = "/content/drive/My Drive/" #@param {type:"string"}
#@markdown > Leave the output_path field empty to use default extraction path (/content).

#@markdown ---
automatically_clear_cell_output = False  # @param{type: "boolean"}
# ================================================================ #

import os, sys, re
from IPython.display import HTML, clear_output


if archive_password == "":
    pass
elif not archive_password == "":
    archive_password = "-p" + archive_password

if output_path == "":
    output_path = "-o/content"
elif output_path == "/content":
    output_path = "-o/content"
else:
    output_path = "-o" + output_path


os.environ['inputFile'] = archive_path
os.environ['inputPassword'] = archive_password
os.environ['outputFile'] = output_path


if archive_path == "":
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ The archive_path field is empty!</h2><br></center>"))
else:
    if archive_type == "zip":
        !7z x "$inputFile" "$inputPassword" "$outputFile"
    elif archive_type == "iso":
        !7z x "$inputFile" "$outputFile"
    elif archive_type == "rar":
        !7z x "$inputFile" "$inputPassword" "$outputFile"
    if archive_type == "tar":
        !7z x "$inputFile" "$outputFile"
    if archive_type == "7z":
        !7z x "$inputFile" "$inputPassword" "$outputFile"
    else:
        display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">❌ More archive format will be added in the future.</h2><br></center>"))


if automatically_clear_cell_output is True:
	clear_output()
else:
	pass